## Графовые нейронные сети (GNNs)

Графовые нейронные сети представляют собой класс нейронных сетей, специально разработанный для работы с графовыми структурами данных. Они могут моделировать зависимости и взаимодействия между сущностями в графе, позволяя учитывать контекст и структуру данных при выполнении задач.

В GNN каждая вершина графа обновляется на основе её соседей, что позволяет модели учить представления вершин, учитывая их окружение. Такие модели широко применяются в задачах, связанных с анализом социальных сетей, биологических сетей, рекомендательных систем и графовых знаний.

## Graph embedding
Эмбеддинги в контексте графовых нейронных сетей представляют собой векторные представления вершин графа. Они извлекаются из GNN и представляют сущности в пространстве низкой размерности. Эти векторы обладают свойством сохранять структурную информацию о графе и могут использоваться для различных задач, таких как предсказание отношений или классификация вершин.

## Тестирование и Валидация

Тестирование и валидация важны для оценки работы графовых нейронных сетей.

Тестирование позволяет оценить обобщающую способность модели на новых данных, которые не участвовали в обучении. Это важно для проверки, насколько хорошо модель обобщает свои знания на новые сценарии.

Валидация используется для подбора оптимальных гиперпараметров модели. Настройка параметров происходит на основе производительности модели на отложенной валидационной выборке.

### Установка зависимостей

Для работы с графовыми нейронными сетями будем использовать [Pykeen](https://pykeen.readthedocs.io/en/stable/). Необходимо импортировать соответствующую библиотеку. В данном шаге мы устанавливаем Pykeen. Этот шаг включает установку библиотеки Pykeen с использованием команды !pip install pykeen. Pykeen предоставляет реализацию различных моделей графовых знаний и инструменты для работы с ними.

In [ ]:
# Установка Pykeen
!pip install pykeen

Далее, мы импортируем библиотеку Pykeen для использования её функциональности в нашем коде.

In [2]:
import pykeen

После выполнения этого шага, мы готовы к использованию Pykeen для создания, обучения и оценки моделей графовых знаний.

## Взаимодействие с графовым датасетом и описание данных

В данной работе мы будем использовать графовый датасет [PharmKG](https://pykeen.readthedocs.io/en/stable/api/pykeen.datasets.PharmKG.html).


Импорт необходимого модуля Pykeen. Мы импортируем модуль datasets из Pykeen для работы с графовыми датасетами.

In [19]:
from pykeen.datasets import PharmKG

Загружаем графовый датасет Nations из Pykeen. В результате выполнения этой команды будет создан объект, представляющий графовый датасет.

In [20]:
# Загрузка графового датасета PharmKG
pharm_dataset = PharmKG()

Мы можем вывести описание датасета, чтобы получить информацию о количестве сущностей, отношений и других характеристиках.

In [21]:
# Просмотр описания датасета
print(pharm_dataset)

INFO:pykeen.datasets.base:reordering columns: ['Entity1_name', 'relationship_type', 'Entity2_name']
INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [691412, 109324, 109324]


PharmKG(num_entities=188296, num_relations=39, create_inverse_triples=False)


#### Получение тренировочных данных
Мы можем получить тренировочные факты, представленные в виде троек сущность-отношение-сущность.
Таким образом, после выполнения этих шагов, у нас есть загруженный графовый датасет, и мы можем использовать его для обучения и тестирования моделей графовых знаний в Pykeen.

In [22]:
# Получение тренировочных фактов (троек сущность-отношение-сущность)
training_data = pharm_dataset.training.mapped_triples
print(training_data[:5])

tensor([[     0,      2, 161966],
        [     0,     21,  26679],
        [     1,     17,  95159],
        [     2,     31,  23633],
        [     3,      3,  65653]])


## Описание базовой задачи

В данном контексте базовой задачей является предсказание отношений между сущностями в графовом датасете. Каждая тройка в графе представляет отношение между двумя сущностями. Например, в социальной сети, где сущности - пользователи и страницы, отношения могут быть "пользователь подписан на страницу".

In [23]:
# Импорт необходимых модулей Pykeen
from pykeen.pipeline import pipeline


#### Выбор модели
Мы выбираем модель для решения задачи предсказания отношений. В данном случае, мы используем модель [TransE](https://paperswithcode.com/method/transe), одну из популярных моделей для этой задачи.

#### Выбор датасета
Мы указываем графовый датасет, с которым будем работать. В данном примере - nations_dataset, который содержит данные о связях в между странами.

#### Количество эпох обучения
Задаем количество эпох обучения (итераций по всему датасету).

Проверяем наличие GPU устройства и вызываем функцию pipeline(), которая запускает процесс обучения модели, тестирования и валидации.

In [24]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [25]:
# Задаем параметры для pipeline
pipeline_results = pipeline(
    model = "TransE",      # Выбираем модель (TransE - одна из моделей для предсказания отношений)
    dataset = pharm_dataset,     # Выбираем графовый датасет (Nations)
    training_kwargs = dict(num_epochs=20),   # Количество эпох обучения
    device = device
)

INFO:pykeen.pipeline.api:Using device: cuda


Training epochs on cuda:0:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=128.


Evaluating on cuda:0:   0%|          | 0.00/109k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 212.34s seconds



Этот код инициирует обучение модели на выбранном датасете, используя выбранную архитектуру (TransE) и указанные параметры. После завершения обучения, результаты выводятся, и можно провести анализ производительности модели.


Печатаем результаты, которые могут включать в себя метрики производительности, потери на тренировочных данных, а также результаты на тестовом и валидационном наборах данных.

In [26]:
# Выводим результаты
print(pipeline_results)

PipelineResult(random_seed=4106710619, model=TransE(
  (loss): MarginRankingLoss(
    (margin_activation): ReLU()
  )
  (interaction): TransEInteraction()
  (entity_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(188296, 50)
    )
  )
  (relation_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(39, 50)
    )
  )
  (weight_regularizers): ModuleList()
), training=TriplesFactory(num_entities=188296, num_relations=39, create_inverse_triples=False, num_triples=874588, path="/root/.data/pykeen/datasets/pharmkg/raw_PharmKG-180k.zip"), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x78a04c44b190>, losses=[0.6803883320935679, 0.2643194973407728, 0.1708590633925165, 0.13345628981509855, 0.11174744682673791, 0.09719923475224668, 0.08738679852241796, 0.07900338306617485, 0.07183585850315324, 0.06749708944032738, 0.06327739458248774, 0.059631001713931264, 0.05703230604490133, 0.05386102187016885, 0.05183096699697915

### Triple Scoring

Импортируем [predict_triples](https://pykeen.readthedocs.io/en/stable/api/pykeen.predict.predict_triples.html)

In [27]:
from pykeen.predict import predict_triples

Рассчитаем оценки для всех validation троек из набора данных, на котором мы обучали модель.

In [28]:
pack = predict_triples(model=pipeline_results.model, triples=pharm_dataset.validation)
df = pack.process(factory=pipeline_results.training).df



```
# This is formatted as code
```

Показаны тройки, получившие наивысший score

In [29]:
df.nlargest(n=5, columns="score")

,head_id,head_label,relation_id,relation_label,tail_id,tail_label,score
12236,55681,cyp19a1,34,V+,55681,cyp19a1,-0.665576
18481,187055,zap,34,V+,187055,zap,-0.665576
40790,158721,sarcoplasmic endoplasmic reticulum calcium atpase,34,V+,158721,sarcoplasmic endoplasmic reticulum calcium atpase,-0.665576
68222,151685,ranbp2,34,V+,151685,ranbp2,-0.665576
75621,27027,aph1a,34,V+,27027,aph1a,-0.665576


В рамках самостоятельной работы попробуйте обучить и протестировать модель на любом другом датасете доступном на https://pykeen.readthedocs.io/en/stable/reference/datasets.html.